<a href="https://colab.research.google.com/github/Viktoriia-kama/ML_hw4-hw16/blob/main/HW10_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV

In [ ]:
!pip install surprise

In [ ]:
data = Dataset.load_builtin('ml-100k')
data

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
models = {
    "SVD": SVD(),
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results = {}

for name, model in models.items():
  print(f"Evaluating {name}...")
  results[name] = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9405  0.9342  0.9385  0.9328  0.9284  0.9349  0.0043  
MAE (testset)     0.7447  0.7366  0.7381  0.7355  0.7293  0.7368  0.0050  
Fit time          3.60    2.99    3.20    1.44    2.11    2.67    0.79    
Test time         0.22    0.49    0.13    0.21    0.21    0.25    0.12    
Evaluating SVD++...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9248  0.9114  0.9185  0.9185  0.9248  0.9196  0.0050  
MAE (testset)     0.7271  0.7148  0.7230  0.7188  0.7238  0.7215  0.0043  
Fit time          27.10   27.43   30.62   28.10   26.96   28.04   1.35    
Test time         4.81    5.70    5.01    4.62    5.28    5.08    0.38    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold

In [ ]:
summary = {}

for name, res in results.items():
  summary[name] = {
       'RMSE Mean': res['test_rmse'].mean(),
        'RMSE Std': res['test_rmse'].std(),
        'MAE Mean': res['test_mae'].mean(),
        'MAE Std': res['test_mae'].std()
  }

results_df = pd.DataFrame(summary).T
results_df

,RMSE Mean,RMSE Std,MAE Mean,MAE Std
SVD,0.934887,0.004280,0.736840,0.004962
SVD++,0.919611,0.004972,0.721513,0.004265
NMF,0.963096,0.003666,0.756958,0.003523


В цьому прикладі, модель SVD++ має найменше середнє значення RMSE та MAE, що означає, що вона найточніша серед трьох. Вона також має найменші значення стандартного відхилення, що означає, що вона найбільш стабільна. Тому, ймовірно, SVD++ є найкращим вибором серед цих трьох моделей.

In [ ]:
# Налаштування параметрів для SVD
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

# Виведення найкращих параметрів та результатів
print("Найкращі параметри:", gs.best_params['rmse'])
print("Найкращий RMSE:", gs.best_score['rmse'])

# Найкраща модель
best_model = gs.best_estimator['rmse']


Найкращі параметри: {'n_factors': 100, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}
Найкращий RMSE: 0.9103497975462005


In [12]:
optimal_params = {
    'n_factors': 100,
    'n_epochs': 40,
    'lr_all': 0.01,
    'reg_all': 0.1
}
model = SVD(**optimal_params)
trainset = data.build_full_trainset()
model.fit(trainset)

models = {
    "SVD": model,
    "SVD++": SVDpp(),
    "NMF": NMF()
}

results1 = {}

for name, model in models.items():
  print(f"Evaluating {name}...")
  results1[name] = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Evaluating SVD...
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9098  0.9096  0.9138  0.9074  0.9089  0.9099  0.0021  
MAE (testset)     0.7197  0.7194  0.7206  0.7168  0.7179  0.7189  0.0014  
Fit time          2.98    7.22    3.22    3.44    2.90    3.95    1.64    
Test time         0.20    0.41    0.12    0.36    0.12    0.24    0.12    
Evaluating SVD++...
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9169  0.9268  0.9157  0.9205  0.9239  0.9208  0.0042  
MAE (testset)     0.7181  0.7252  0.7191  0.7221  0.7252  0.7220  0.0030  
Fit time          26.82   28.68   26.83   27.45   26.91   27.34   0.71    
Test time         4.74    4.59    5.43    4.57    6.02    5.07    0.57    
Evaluating NMF...
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold

In [11]:
summary1 = {}
for name, res in results1.items():
  summary1[name] = {
        'RMSE Mean': np.mean(res['test_rmse']),
        'RMSE Std': np.std(res['test_rmse']),
        'MAE Mean': np.mean(res['test_mae']),
        'MAE Std': np.std(res['test_mae']),
    }

results_df1 = pd.DataFrame(summary1).T
results_df1

,RMSE Mean,RMSE Std,MAE Mean,MAE Std
SVD,0.942280,0.004400,0.740338,0.003872
SVD++,0.918601,0.004152,0.720500,0.003255
NMF,0.962187,0.004734,0.756145,0.003942


Порівнявши результати метрик, роблю висновок, що за допомогою GridSearchCV не було знайдено параметрів, які б покращили результат роботи алгоритму SVD. Метрики отримані після першого тренування є кращими, ніж після GridSearchCV

У результаті, найбільш оптимальним алгоритмом є SVD++, оскільки його метрики є найнижчими